In [2]:
import json
import os
import re
import numpy as np 
import pathlib as pb
import pandas as pd
import matplotlib.pyplot as plt 
import scipy.stats as stats 
import seaborn as sns
from scipy.stats import f_oneway
from pathlib import Path
from statannot import add_stat_annotation


In [3]:
# Returns the specific frames where the participant was on the right side, number of times participant went to right side, total number of frames where the participant was on the right side, and whether participant was still on the right side at the end of the game

def right_side(json_file):
    new_list = " "
    on_other_side = False
    start_frame = 0
    end_frame = 0
    total_frames = 0
    still_on_other_side = False
    frames = json_file['frames']
    right_side_times = 0
    ai_alive = True
    participant_alive = True
    ai_died = 0
    participant_died = 0
    for frame in frames:
        if frame['player_position'] > 400 and on_other_side == False:
            start_frame = frame['frame_number']
            on_other_side = True
        if frame['player_position'] < 400 and on_other_side == True:
            on_other_side = False
            end_frame = frame['frame_number']
            new_list += str({'start_frame': start_frame, 'end_frame': end_frame}) 
            new_list += ', '
            total_frames = total_frames + (end_frame - start_frame)
            right_side_times += 1
        if frame['ai_lives'] == 0 and ai_alive == True:
            ai_died = frame['frame_number']
            ai_alive = False
        if frame['player_lives'] == 0 and participant_alive == True:
            participant_died = frame['frame_number']
            participant_alive = False
    if (on_other_side):
        end_frame = frame['frame_number']
        right_side_times += 1
        new_list += str({'start_frame': start_frame, 'end_frame': end_frame})
        new_list += ', '
        total_frames = total_frames + (end_frame - start_frame)
    final_ai_score = frame['ai_score']
    final_part_score = frame['player_score']
    total_score = final_ai_score + final_part_score
    final_ai_lives = frame['ai_lives']
    final_part_lives = frame['player_lives']
    percentage_of_game_played = (participant_died / frame['frame_number']) * 100
    return {'Participant Visits': new_list, 'Participant Number of Visits': right_side_times,'Participant Total Frames on Right Side': total_frames, 'Final AI Score': final_ai_score, 'Final Participant Score': final_part_score, 'Total Score':total_score, 'Final AI Lives' : final_ai_lives, 'Final Participant Lives': final_part_lives, 'Frame of Participant Death (0 if not completely dead)': participant_died, 'Frame of AI Death (0 if not completely dead)': ai_died, 'Percentage of Game Played (0 if not completely dead)': percentage_of_game_played}

In [4]:
# Returns the specific frames where the coplayer was on the left side, number of times coplayer went to left side, total number of frames where the coplayer was on the left side, and whether coplayer was still on the left side at the end of the game

def left_side(json_file):
    new_list = " "
    on_other_side = False
    start_frame = 0
    end_frame = 0
    total_frames = 0
    end_frame_number = 0
    still_on_other_side = False
    frames = json_file['frames']
    left_side_times = 0
    for frame in frames:
        if frame['ai_position'] < 400 and on_other_side == False: #if AI is on the left side
            start_frame = frame['frame_number']
            on_other_side = True
        elif frame['ai_position'] > 400 and on_other_side == True: #if AI moves back to the right side
            on_other_side = False
            end_frame = frame['frame_number']
            new_list += str({'start_frame': start_frame, 'end_frame': end_frame}) 
            new_list += ', '            
            total_frames = total_frames + (end_frame - start_frame)
            left_side_times += 1
    if (on_other_side):
        end_frame = frame['frame_number']
        left_side_times += 1
        new_list += str({'start_frame': start_frame, 'end_frame': end_frame})
        new_list += ', '
        total_frames = total_frames + (end_frame - start_frame)
    end_frame_number = frame['frame_number']
    return {'Co-player Visits': new_list, 'Co-player Number of Visits': left_side_times, 'Co-player Total Frames on Left Side': total_frames, 'End Frame':end_frame_number}

In [5]:
def part_enemies_count(json_file):
    participant_count = 0
    events = json_file['events']
    for event in events:
        if event['killer'] == 'PLAYER' and event['killed'] == 'RIGHT':
            participant_count += 1
    return participant_count


In [6]:
def co_enemies_count(json_file):
    coplayer_count = 0
    events = json_file['events']
    for event in events:
        if event['killer'] == 'AI' and event['killed'] == 'LEFT':
            coplayer_count += 1
    return coplayer_count


In [7]:
path_name = "/Users/zoehsu/Desktop/IMG/ExpZ_V2/game_logs"
folder = os.listdir(path_name)
complete_chart = []
for file in folder:
  if file != ".DS_Store":
    file = f"{path_name}/{file}" #have to do this because you're in a diff directory
    with open(file, encoding='windows-1252', mode='r') as file:
      data = json.load(file)
      participant_enemies_count = part_enemies_count(data)
      coplayer_enemies_count = co_enemies_count(data)
      right_side_info = right_side(data)
      left_side_info = left_side(data)
      right_side_info.update(left_side_info)
      right_side_info.update({"Enemies killed by participant": participant_enemies_count})
      right_side_info.update({"Enemies killed by coplayer": coplayer_enemies_count})
      right_side_info.update({"Participant ID":Path(file.name).stem.split("_")[0][1:]})
      right_side_info.update({"File name":Path(file.name).stem + '.json'})
      complete_chart.append(right_side_info)
      if 'voff_m1' in Path(file.name).stem:
        right_side_info.update({"Mode" : 1 })
      elif 'voff_m2' in Path(file.name).stem:
        right_side_info.update({"Mode" : 2 })
df = pd.DataFrame(complete_chart)
df.to_csv (r'/Users/zoehsu/Desktop/alldata.csv', index = True, header=True)

KeyboardInterrupt: 

In [ ]:
full_data = pd.read_csv('/Users/zoehsu/Desktop/alldata.csv')

In [ ]:
expz_data = pd.read_csv('/Users/zoehsu/Desktop/IMG/ExpZ_V2/ExpZ2_ Participants - ParticipantRecords.csv')
qualtrics_data = pd.read_csv('/Users/zoehsu/Desktop/IMG/ExpZ_V2/ExpZ_Qualtrics_07252021_180p_stacked.xlsx - Used.csv')

# Combining ExpZ data with Qualtrics data
expz_qual_dataframe = pd.merge(expz_data, qualtrics_data, left_on='Prolific ID', right_on='PROLIFIC_PID') 

# Creating dataframe with Mode 1 participants
mode1_grouped = full_data.groupby('Mode')
mode1_data = mode1_grouped.get_group(1)

# Creating dataframe with Mode 2 participants
mode2_grouped = full_data.groupby('Mode')
mode2_data = mode2_grouped.get_group(2)

# Combining ExpZ&Qualtrics data with Mode 1 data
semifinal_dataframe = pd.merge(expz_qual_dataframe, mode1_data, left_on='Prolific ID', right_on='Participant ID') 

# Combining ExpZ&Qualtrics&Mode1 with Mode 2 data
new_dataframe = pd.merge(semifinal_dataframe, mode2_data, left_on='Prolific ID', right_on='Participant ID') 
new_dataframe.rename(columns={"Group_x":"Group"}, inplace = True) # Renaming group_x to group

# replacing group #s with group identity names
new_dataframe["Group"].replace({1:"AI", 2:"Human", 4: "Computer"}, inplace = True)

# Creating the master spreadsheet
new_dataframe.to_csv (r'/Users/zoehsu/Desktop/FINAL.csv', index = True, header=True)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/zoehsu/Desktop/IMG/ExpZ2_ Participants - ParticipantRecords.csv'

In [ ]:
enemies_exclusion = new_dataframe[ (new_dataframe["Enemies killed by coplayer_x"] == 0) & (new_dataframe["First"] == 'Early')]

enemies_exclusion

NameError: name 'new_dataframe' is not defined

In [ ]:
enemies_exclusion_early2nd = new_dataframe[ (new_dataframe["Enemies killed by coplayer_x"] == 0) & (new_dataframe["First"] == 'Late')]

enemies_exclusion_early2nd


NameError: name 'new_dataframe' is not defined